In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/kazakhstan-respa-final-day-2-late-competition/sample_submission.csv
/kaggle/input/kazakhstan-respa-final-day-2-late-competition/test_sentences.csv
/kaggle/input/kazakhstan-respa-final-day-2-late-competition/train_sentences.csv
/kaggle/input/kazakhstan-respa-final-day-2-late-competition/train_timeseries.csv


In [2]:
train = pd.read_csv('/kaggle/input/kazakhstan-respa-final-day-2-late-competition/train_sentences.csv')
train

,submitted_date,sentences,full_categories
0,2000-05-25 01:07:16,We give a summary of recent results on spatial...,['astro-ph']
1,2000-09-29 17:10:21,We study properties of the oscillation effects...,"['hep-ph', 'astro-ph', 'hep-ex']"
2,2000-10-04 09:40:11,Motivated by observations of supernova SN 1987...,['astro-ph']
3,2000-07-02 04:10:19,The abundance of oxygen was determined for sel...,['astro-ph']
4,2000-09-04 16:35:10,"The 0th, 1st and 2nd derivatives of a ``Fermat...",['astro-ph']
...,...,...,...
37172,2025-01-31 17:35:21,Causal Bayesian networks are 'causal' models s...,"['stat.ML', 'cs.AI', 'cs.LG', 'math.ST', 'stat..."
37173,2025-03-21 14:27:41,The Friedman test has been extensively applied...,"['stat.ME', 'math.ST', 'stat.TH']"
37174,2025-02-21 10:08:28,"In this paper, we propose a novel Bayesian app...","['math.ST', 'stat.TH']"
37175,2025-04-04 14:17:45,We consider a classical First-order Vector Aut...,"['math.ST', 'stat.TH']"


In [3]:
train['submitted_date'] = pd.to_datetime(train['submitted_date'])

In [4]:
train['submitted_date'] = train['submitted_date'].dt.year

In [5]:
train

,submitted_date,sentences,full_categories
0,2000,We give a summary of recent results on spatial...,['astro-ph']
1,2000,We study properties of the oscillation effects...,"['hep-ph', 'astro-ph', 'hep-ex']"
2,2000,Motivated by observations of supernova SN 1987...,['astro-ph']
3,2000,The abundance of oxygen was determined for sel...,['astro-ph']
4,2000,"The 0th, 1st and 2nd derivatives of a ``Fermat...",['astro-ph']
...,...,...,...
37172,2025,Causal Bayesian networks are 'causal' models s...,"['stat.ML', 'cs.AI', 'cs.LG', 'math.ST', 'stat..."
37173,2025,The Friedman test has been extensively applied...,"['stat.ME', 'math.ST', 'stat.TH']"
37174,2025,"In this paper, we propose a novel Bayesian app...","['math.ST', 'stat.TH']"
37175,2025,We consider a classical First-order Vector Aut...,"['math.ST', 'stat.TH']"


In [6]:
import numpy as np
import pandas as pd
import time
import datetime
import gc
import random
from nltk.corpus import stopwords
import re

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler,random_split
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import transformers
from transformers import BertForSequenceClassification, BertConfig,BertTokenizer,get_linear_schedule_with_warmup, DistilBertForSequenceClassification, DistilBertTokenizer

2025-06-11 05:14:37.898972: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749618878.094636      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749618878.150268      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [7]:
sw = stopwords.words('english')

def clean_text(text):
    
    text = text.lower()
    
    text = re.sub(r"[^a-zA-Z?.!,¿]+", " ", text) # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")

    text = re.sub(r"http\S+", "",text) #Removing URLs 
    #text = re.sub(r"http", "",text)
    
    html=re.compile(r'<.*?>') 
    
    text = html.sub(r'',text) #Removing html tags
    
    punctuations = '@#!?+&*[]-%.:/();$=><|{}^' + "'`" + '_'
    for p in punctuations:
        text = text.replace(p,'') #Removing punctuations
        
    text = [word.lower() for word in text.split() if word.lower() not in sw]
    
    text = " ".join(text) #removing stopwords
    
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text) #Removing emojis
    
    return text

In [8]:
tweets = train['sentences'].apply(lambda x: clean_text(x))

labels = train['submitted_date']

In [9]:
from sklearn.preprocessing import LabelEncoder
import numpy as np

# предположим, у вас train['submitted_date'] → year_ids
le = LabelEncoder()
year_ids = le.fit_transform(train['submitted_date'])
print("Уникальные годовые метки:", np.unique(year_ids))
print("Классы LabelEncoder:", le.classes_)
n_classes = len(le.classes_)
print("num_labels:", n_classes)

Уникальные годовые метки: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25]
Классы LabelEncoder: [2000 2001 2002 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013
 2014 2015 2016 2017 2018 2019 2020 2021 2022 2023 2024 2025]
num_labels: 26


In [10]:
year_ids[:100]

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  5,
        5,  5,  5,  5,  5,  5,  5,  5,  5,  6,  6,  6,  6,  6,  6,  6,  6,
        6,  6,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  8,  8,  8,  8,  8,
        8,  8,  8,  8,  8, 19,  3,  3,  3,  4,  4,  4,  5,  5,  5])

In [11]:
labels[:100]

0     2000
1     2000
2     2000
3     2000
4     2000
      ... 
95    2004
96    2004
97    2005
98    2005
99    2005
Name: submitted_date, Length: 100, dtype: int32

In [12]:
labels = year_ids

In [13]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [14]:
modelname = "distilbert-base-uncased"
# modelname = 'distilroberta-base'

In [15]:
tokenizer = DistilBertTokenizer.from_pretrained(modelname, do_lower_case=True)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [16]:
print(' Original: ', tweets[0])

# Print the sentence split into tokens.
print('Tokenized: ', tokenizer.tokenize(tweets[0]))

# Print the sentence mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(tweets[0])))

 Original:  give summary recent results spatial velocity biases cosmological models progress numerical techniques made possible simulate halos large volumes accuracy halos survive dense environments groups clusters galaxies
Tokenized:  ['give', 'summary', 'recent', 'results', 'spatial', 'velocity', 'bias', '##es', 'co', '##smo', '##logical', 'models', 'progress', 'numerical', 'techniques', 'made', 'possible', 'simulate', 'halo', '##s', 'large', 'volumes', 'accuracy', 'halo', '##s', 'survive', 'dense', 'environments', 'groups', 'clusters', 'galaxies']
Token IDs:  [2507, 12654, 3522, 3463, 13589, 10146, 13827, 2229, 2522, 25855, 9966, 4275, 5082, 15973, 5461, 2081, 2825, 26633, 17201, 2015, 2312, 6702, 10640, 17201, 2015, 5788, 9742, 10058, 2967, 12906, 21706]


In [17]:
max_len = 0

# For every sentence...
for sent in tweets:

    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(sent, add_special_tokens=True)

    # Update the maximum sentence length.
    max_len = max(max_len, len(input_ids))

print('Max sentence length: ', max_len)

Max sentence length:  275


In [18]:
max_len = 256

In [19]:
input_ids = []
attention_masks = []

# For every tweet...
for tweet in tweets:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        tweet,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = max_len,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

# Print sentence 0, now as a list of IDs.
print('Original: ', tweets[0])
print('Token IDs:', input_ids[0])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:2700: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Original:  give summary recent results spatial velocity biases cosmological models progress numerical techniques made possible simulate halos large volumes accuracy halos survive dense environments groups clusters galaxies
Token IDs: tensor([  101,  2507, 12654,  3522,  3463, 13589, 10146, 13827,  2229,  2522,
        25855,  9966,  4275,  5082, 15973,  5461,  2081,  2825, 26633, 17201,
         2015,  2312,  6702, 10640, 17201,  2015,  5788,  9742, 10058,  2967,
        12906, 21706,   102,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0, 

In [20]:
dataset = TensorDataset(input_ids, attention_masks, labels)

# Create a 90-10 train-validation split.

# Calculate the number of samples to include in each set.
train_size = int(0.8 * len(dataset))
#val_size = int(0.2 * len(dataset))
val_size = len(dataset)  - train_size

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

29,741 training samples
7,436 validation samples


In [21]:
# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 16

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [22]:
# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model = DistilBertForSequenceClassification.from_pretrained(
    modelname, # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = len(labels.unique()), # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

# if device == "cuda:0":
# # Tell pytorch to run this model on the GPU.
#     model = model.cuda()
model = model.to(device)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
optimizer = torch.optim.AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

In [24]:
# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.
epochs = 4

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [25]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [26]:
def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
from tqdm.auto import tqdm

seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
training_stats = []

for epoch_i in range(epochs):
    print(f"\n======== Epoch {epoch_i+1} / {epochs} ========")
    
    # TRAINING
    model.train()
    total_train_loss = 0.0
    
    train_iter = tqdm(train_dataloader, desc="Train", unit="batch")
    for batch in train_iter:
        b_input_ids, b_input_mask, b_labels = [t.to(device) for t in batch]
        b_labels = b_labels.long()
        
        optimizer.zero_grad()
        outputs = model(
            b_input_ids,
            attention_mask=b_input_mask,
            labels=b_labels
        )
        loss = outputs.loss
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        
        batch_loss = loss.item()
        total_train_loss += batch_loss
        
        # обновляем tqdm-бар: показываем текущий batch_loss
        train_iter.set_postfix(train_loss=batch_loss)

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.
    print("")
    print("Running Validation...")
    t0 = time.time()
    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()
    # Tracking variables 
    total_eval_accuracy = 0
    best_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0
    # Evaluate data for one epoch
    for batch in validation_dataloader:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        
            output= model(b_input_ids, 
                                   attention_mask=b_input_mask,
                                   labels=b_labels)
        loss = output.loss
        total_eval_loss += loss.item()
        # Move logits and labels to CPU if we are using GPU
        logits = output.logits
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))
    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    # Measure how long the validation run took.
    if avg_val_accuracy > best_eval_accuracy:
        torch.save(model, 'bert_model')
        best_eval_accuracy = avg_val_accuracy
    #print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    #print("  Validation took: {:}".format(validation_time))
    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
        }
    )
print("")
print("Training complete!")


======== Epoch 1 / 4 ========


Train:   0%|          | 0/1859 [00:00<?, ?batch/s]

In [ ]:
testpath = '/kaggle/input/kazakhstan-respa-final-day-2-late-competition/test_sentences.csv'
df_test = pd.read_csv(testpath)
df_test['text1'] = df_test['first_sentence'].apply(lambda x:clean_text(x))
test_tweets1 = df_test['text1'].values
df_test['text2'] = df_test['second_sentence'].apply(lambda x:clean_text(x))
test_tweets2 = df_test['text2'].values

In [ ]:
test_input_ids1 = []
test_attention_masks1 = []
for tweet in test_tweets1:
    encoded_dict = tokenizer.encode_plus(
                        tweet,                     
                        add_special_tokens = True, 
                        max_length = max_len,           
                        pad_to_max_length = True,
                        return_attention_mask = True,
                        return_tensors = 'pt',
                   )
    test_input_ids1.append(encoded_dict['input_ids'])
    test_attention_masks1.append(encoded_dict['attention_mask'])
test_input_ids1 = torch.cat(test_input_ids1, dim=0)
test_attention_masks1 = torch.cat(test_attention_masks1, dim=0)

In [ ]:
test_input_ids2 = []
test_attention_masks2 = []
for tweet in test_tweets2:
    encoded_dict = tokenizer.encode_plus(
                        tweet,                     
                        add_special_tokens = True, 
                        max_length = max_len,           
                        pad_to_max_length = True,
                        return_attention_mask = True,
                        return_tensors = 'pt',
                   )
    test_input_ids2.append(encoded_dict['input_ids'])
    test_attention_masks2.append(encoded_dict['attention_mask'])
test_input_ids2 = torch.cat(test_input_ids2, dim=0)
test_attention_masks2 = torch.cat(test_attention_masks2, dim=0)

In [ ]:
test_dataset1 = TensorDataset(test_input_ids1, test_attention_masks1)
test_dataloader1 = DataLoader(
            test_dataset1, # The validation samples.
            sampler = SequentialSampler(test_dataset1), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [ ]:
test_dataset2 = TensorDataset(test_input_ids2, test_attention_masks2)
test_dataloader2 = DataLoader(
            test_dataset2, # The validation samples.
            sampler = SequentialSampler(test_dataset2), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [ ]:
model = torch.load('bert_model')

In [ ]:
predictions1 = []
for batch in test_dataloader1:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        with torch.no_grad():        
            output= model(b_input_ids, 
                                   attention_mask=b_input_mask)
            logits = output.logits
            logits = logits.detach().cpu().numpy()
            pred_flat = np.argmax(logits, axis=1).flatten()
            
            predictions1.extend(list(pred_flat))

In [ ]:
predictions1[:10]

In [ ]:
predictions2 = []
for batch in test_dataloader2:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        with torch.no_grad():        
            output= model(b_input_ids, 
                                   attention_mask=b_input_mask)
            logits = output.logits
            logits = logits.detach().cpu().numpy()
            pred_flat = np.argmax(logits, axis=1).flatten()
            
            predictions2.extend(list(pred_flat))

In [ ]:
predictions2[:10]

In [ ]:
submission = pd.read_csv('/kaggle/input/kazakhstan-respa-final-day-2-late-competition/sample_submission.csv')
submission

In [ ]:
preds = []
for f, s in zip(predictions1, predictions2):
    if f > s:
        preds.append(0)
    else:
        preds.append(1)

In [ ]:
submission['label'] = preds
submission

In [ ]:
submission.to_csv('submission111.csv', index=False)